<a href="https://colab.research.google.com/github/siddhu-pikachu/coin_detection/blob/main/yolonas_custom_coin_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Jun 25 13:35:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q super-gradients==3.1.1
!pip install -q roboflow
!pip install -q supervision

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 1.1.0 requires pyparsing==2.4.7, but you have pyparsing 2.4.5 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
super-gradients 3.1.1 requires pyparsing==2.4.5, but you have pyparsing 2.4.7 which is incompatible.


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
import torch

In [ ]:
f"{HOME}/data"

'/content/data'

In [ ]:
%cd {HOME}

import roboflow
from roboflow import Roboflow

roboflow.login()

rf = Roboflow()

project = rf.workspace("coindetection-iu1te").project("coin-detector-and-counter")
dataset = project.version(2).download("yolov5")

/content
You are already logged into Roboflow. To make a different login, run roboflow.login(force=True).
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to coin-detector-and-counter-2 in yolov5pytorch:: 100%|██████████| 5486/5486 [00:03<00:00, 1580.57it/s]


In [ ]:
LOCATION = dataset.location
print("location:", LOCATION)
CLASSES = sorted(project.classes.keys())
print("classes:", CLASSES)

location: /content/coin-detector-and-counter-2
classes: ['10_coin', '1_coin', '2_coin', '5_coin']


In [ ]:
MODEL_ARCH = 'yolo_nas_s'
BATCH_SIZE = 8
MAX_EPOCHS = 15
CHECKPOINT_DIR = f'{HOME}/checkpoints'
EXPERIMENT_NAME = project.name.lower().replace(" ", "_")

In [ ]:
from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

[2023-06-25 13:37:42] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


Setuptools is replacing distutils.
[2023-06-25 13:37:47] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-06-25 13:37:47] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-06-25 13:37:47] WARNING - export.py - Failed to import pytorch_quantization
[2023-06-25 13:37:47] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


In [ ]:
dataset_params = {
    'data_dir': LOCATION,
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': CLASSES
}

In [ ]:
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

Caching annotations: 100%|██████████| 2385/2385 [00:00<00:00, 5132.91it/s]
[2023-06-25 13:37:48] WARNING - detection_dataset.py - Found 3 invalid bbox that were ignored. For more information, please set `show_all_warnings=True`.
Caching annotations:   0%|          | 0/123 [00:00<?, ?it/s]

In [ ]:
train_data.dataset.transforms

[DetectionMosaic('additional_samples_count': 3, 'non_empty_targets': False, 'prob': 1.0, 'input_dim': [640, 640], 'enable_mosaic': True, 'border_value': 114),
 DetectionRandomAffine('additional_samples_count': 0, 'non_empty_targets': False, 'degrees': 10.0, 'translate': 0.1, 'scale': [0.1, 2], 'shear': 2.0, 'target_size': [640, 640], 'enable': True, 'filter_box_candidates': True, 'wh_thr': 2, 'ar_thr': 20, 'area_thr': 0.1, 'border_value': 114),
 DetectionMixup('additional_samples_count': 1, 'non_empty_targets': True, 'input_dim': [640, 640], 'mixup_scale': [0.5, 1.5], 'prob': 1.0, 'enable_mixup': True, 'flip_prob': 0.5, 'border_value': 114),
 DetectionHSV('additional_samples_count': 0, 'non_empty_targets': False, 'prob': 1.0, 'hgain': 5, 'sgain': 30, 'vgain': 30, 'bgr_channels': (0, 1, 2), '_additional_channels_warned': False),
 DetectionHorizontalFlip('additional_samples_count': 0, 'non_empty_targets': False, 'prob': 0.5, 'max_targets': 120),
 DetectionPaddedRescale('swap': (2, 0, 1),

### Instantiate the model

In [ ]:
from super_gradients.training import models

model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    pretrained_weights="coco"
)

Caching annotations: 100%|██████████| 123/123 [00:00<00:00, 4647.41it/s]
[2023-06-25 13:37:48] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
Downloading: "https://sghub.deci.ai/models/yolo_nas_s_coco.pth" to /root/.cache/torch/hub/checkpoints/yolo_nas_s_coco.pth


  0%|          | 0.00/73.1M [00:00<?, ?B/s]

### Define metrics and training parameters

In [ ]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

### Train the model

In [ ]:
trainer.train(
    model=model,
    training_params=train_params,
    train_loader=train_data,
    valid_loader=val_data
)

[2023-06-25 13:37:53] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}
[2023-06-25 13:37:58] INFO - utils.py - NumExpr defaulting to 2 threads.


The console stream is now moved to /content/checkpoints/coin_detector_and_counter/console_Jun25_13_37_58.txt


[2023-06-25 13:38:04] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (1 available on the machine)
    - Dataset size:                 2379       (len(train_set))
    - Batch size per GPU:           8          (batch_size)
    - Batch Accumulate:             1          (batch_accumulate)
    - Total batch size:             8          (num_gpus * batch_size)
    - Effective Batch size:         8          (num_gpus * batch_size * batch_accumulate)
    - Iterations per epoch:         297        (len(train_loader))
    - Gradient updates per epoch:   297        (len(train_loader) / batch_accumulate)

[2023-06-25 13:38:04] INFO - sg_trainer.py - Started training for 15 epochs (0/14)

Validation epoch 0: 100%|██████████| 29/29 [00:05<00:00,  5.53it/s]


SUMMARY OF EPOCH 0
├── Training
│   ├── Ppyoloeloss/loss = 4.0676
│   ├── Ppyoloeloss/loss_cls = 2.733
│   ├── Ppyoloeloss/loss_dfl = 1.5042
│   └── Ppyoloeloss/loss_iou = 0.233
└── Validation
    ├── F1@0.50 = 0.0009
    ├── Map@0.50 = 0.0073
    ├── Ppyoloeloss/loss = 3.4611
    ├── Ppyoloeloss/loss_cls = 2.7857
    ├── Ppyoloeloss/loss_dfl = 0.8939
    ├── Ppyoloeloss/loss_iou = 0.0914
    ├── Precision@0.50 = 0.0005
    └── Recall@0.50 = 0.027



[2023-06-25 13:41:39] INFO - base_sg_logger.py - Checkpoint saved in /content/checkpoints/coin_detector_and_counter/ckpt_best.pth
[2023-06-25 13:41:39] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.007330538239330053
Validation epoch 1: 100%|██████████| 29/29 [00:05<00:00,  5.55it/s]


...

[2023-06-25 14:24:16] INFO - base_sg_logger.py - Checkpoint saved in /content/checkpoints/coin_detector_and_counter/ckpt_best.pth
[2023-06-25 14:24:16] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.72493577003479
Validation epoch 13: 100%|██████████| 29/29 [00:06<00:00,  4.46it/s]


SUMMARY OF EPOCH 13
├── Training
│   ├── Ppyoloeloss/loss = 1.6634
│   │   ├── Best until now = 1.6997 (↘ -0.0363)
│   │   └── Epoch N-1      = 1.6997 (↘ -0.0363)
│   ├── Ppyoloeloss/loss_cls = 0.9576
│   │   ├── Best until now = 0.9751 (↘ -0.0175)
│   │   └── Epoch N-1      = 0.9751 (↘ -0.0175)
│   ├── Ppyoloeloss/loss_dfl = 0.8631
│   │   ├── Best until now = 0.8824 (↘ -0.0192)
│   │   └── Epoch N-1      = 0.8824 (↘ -0.0192)
│   └── Ppyoloeloss/loss_iou = 0.1097
│       ├── Best until now = 0.1134 (↘ -0.0037)
│       └── Epoch N-1      = 0.1134 (↘ -0.0037)
└── Validation
    ├── F1@0.50 = 0.1603
    │   ├── Best until now = 0.1389 (↗ 0.0214)
    │   └── Epoch N-1      = 0.1185 (↗ 0.0419)
    ├── Map@0.50 = 0.6903
    │   ├── Best until now = 0.7249 (↘ -0.0346)
    │   └── Epoch N-1      = 0.7249 (↘ -0.0346)
    ├── Ppyoloeloss/loss = 1.4796
    │   ├── Best until now = 1.5137 (↘ -0.034)
    │   └── Epoch N-1      = 1.5163 (↘ -0.0367)
    ├── Ppyoloeloss/loss_cls = 0.895
    │   ├── B

Validation epoch 14: 100%|██████████| 29/29 [00:06<00:00,  4.46it/s]


SUMMARY OF EPOCH 14
├── Training
│   ├── Ppyoloeloss/loss = 1.6522
│   │   ├── Best until now = 1.6634 (↘ -0.0113)
│   │   └── Epoch N-1      = 1.6634 (↘ -0.0113)
│   ├── Ppyoloeloss/loss_cls = 0.9482
│   │   ├── Best until now = 0.9576 (↘ -0.0094)
│   │   └── Epoch N-1      = 0.9576 (↘ -0.0094)
│   ├── Ppyoloeloss/loss_dfl = 0.8654
│   │   ├── Best until now = 0.8631 (↗ 0.0022)
│   │   └── Epoch N-1      = 0.8631 (↗ 0.0022)
│   └── Ppyoloeloss/loss_iou = 0.1085
│       ├── Best until now = 0.1097 (↘ -0.0012)
│       └── Epoch N-1      = 0.1097 (↘ -0.0012)
└── Validation
    ├── F1@0.50 = 0.1005
    │   ├── Best until now = 0.1603 (↘ -0.0598)
    │   └── Epoch N-1      = 0.1603 (↘ -0.0598)
    ├── Map@0.50 = 0.7159
    │   ├── Best until now = 0.7249 (↘ -0.009)
    │   └── Epoch N-1      = 0.6903 (↗ 0.0256)
    ├── Ppyoloeloss/loss = 1.5485
    │   ├── Best until now = 1.4796 (↗ 0.0688)
    │   └── Epoch N-1      = 1.4796 (↗ 0.0688)
    ├── Ppyoloeloss/loss_cls = 0.9322
    │   ├── Bes

[2023-06-25 14:31:31] INFO - sg_trainer.py - RUNNING ADDITIONAL TEST ON THE AVERAGED MODEL...
Validation epoch 15:  97%|█████████▋| 28/29 [00:05<00:00,  6.27it/s]

### Analyze training metrics


In [ ]:
%load_ext tensorboard
%tensorboard --logdir {CHECKPOINT_DIR}/{EXPERIMENT_NAME}

<IPython.core.display.Javascript object>

Validation epoch 15: 100%|██████████| 29/29 [00:05<00:00,  5.32it/s]
[2023-06-25 14:31:37] INFO - base_sg_logger.py - [CLEANUP] - Successfully stopped system monitoring process


SUMMARY OF EPOCH 15
├── Training
│   ├── Ppyoloeloss/loss = 1.6522
│   │   ├── Best until now = 1.6634 (↘ -0.0113)
│   │   └── Epoch N-1      = 1.6634 (↘ -0.0113)
│   ├── Ppyoloeloss/loss_cls = 0.9482
│   │   ├── Best until now = 0.9576 (↘ -0.0094)
│   │   └── Epoch N-1      = 0.9576 (↘ -0.0094)
│   ├── Ppyoloeloss/loss_dfl = 0.8654
│   │   ├── Best until now = 0.8631 (↗ 0.0022)
│   │   └── Epoch N-1      = 0.8631 (↗ 0.0022)
│   └── Ppyoloeloss/loss_iou = 0.1085
│       ├── Best until now = 0.1097 (↘ -0.0012)
│       └── Epoch N-1      = 0.1097 (↘ -0.0012)
└── Validation
    ├── F1@0.50 = 0.1668
    │   ├── Best until now = 0.1603 (↗ 0.0065)
    │   └── Epoch N-1      = 0.1005 (↗ 0.0663)
    ├── Map@0.50 = 0.6928
    │   ├── Best until now = 0.7249 (↘ -0.0321)
    │   └── Epoch N-1      = 0.7159 (↘ -0.0231)
    ├── Ppyoloeloss/loss = 1.4593
    │   ├── Best until now = 1.4796 (↘ -0.0204)
    │   └── Epoch N-1      = 1.5485 (↘ -0.0892)
    ├── Ppyoloeloss/loss_cls = 0.8783
    │   ├── B

## Zip and download fine-tuned model

In [ ]:
!zip -r yolo_nas.zip {CHECKPOINT_DIR}/{EXPERIMENT_NAME}

updating: content/checkpoints/coin_detector_and_counter/ (stored 0%)
updating: content/checkpoints/coin_detector_and_counter/events.out.tfevents.1687703497.abb6b739317f.5179.1 (deflated 42%)
updating: content/checkpoints/coin_detector_and_counter/ckpt_latest.pth (deflated 8%)
updating: content/checkpoints/coin_detector_and_counter/events.out.tfevents.1687700278.abb6b739317f.5179.0 (deflated 82%)
updating: content/checkpoints/coin_detector_and_counter/console_Jun25_13_37_58.txt (deflated 95%)
updating: content/checkpoints/coin_detector_and_counter/experiment_logs_Jun25_13_37_58.txt (deflated 73%)
updating: content/checkpoints/coin_detector_and_counter/logs_Jun25_13_37_58.txt (deflated 78%)
updating: content/checkpoints/coin_detector_and_counter/average_model.pth (deflated 8%)
updating: content/checkpoints/coin_detector_and_counter/ckpt_best.pth

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# if you experience 'NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968' error, run code below 👇

import locale
locale.getpreferredencoding = lambda: "UTF-8"

## Load trained model

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    checkpoint_path=f"/content/drive/MyDrive/saved_models/{EXPERIMENT_NAME}/average_model.pth"
).to(DEVICE)

FileNotFoundError: ignored

## Evaluate trained model

In [ ]:
trainer.test(
    model=best_model,
    test_loader=test_data,
    test_metrics_list=DetectionMetrics_050(
        score_thres=0.1,
        top_k_predictions=300,
        num_cls=len(dataset_params['classes']),
        normalize_targets=True,
        post_prediction_callback=PPYoloEPostPredictionCallback(
            score_threshold=0.01,
            nms_top_k=1000,
            max_predictions=300,
            nms_threshold=0.7
        )
    )
)

# Test: 100%|██████████| 16/16 [00:03<00:00,  4.94it/s]
# {'PPYoloELoss/loss_cls': 0.95609295,
#  'PPYoloELoss/loss_iou': 0.08095996,
#  'PPYoloELoss/loss_dfl': 0.73067665,
#  'PPYoloELoss/loss': 1.5238311,
#  'Precision@0.50': tensor(0.0738),
#  'Recall@0.50': tensor(1.),
#  'mAP@0.50': tensor(0.6059),
#  'F1@0.50': tensor(0.1365)}

### Inference with trained model

In [ ]:
import supervision as sv

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{dataset.location}/test/images",
    annotations_directory_path=f"{dataset.location}/test/labels",
    data_yaml_path=f"{dataset.location}/data.yaml",
    force_masks=False
)

In [ ]:
import supervision as sv

CONFIDENCE_TRESHOLD = 0.5

predictions = {}

for image_name, image in ds.images.items():
    result = list(best_model.predict(image, conf=CONFIDENCE_TRESHOLD))[0]
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections

### Visualize inference results

In [ ]:
import random
random.seed(10)

In [ ]:
import supervision as sv

MAX_IMAGE_COUNT = 5

n = min(MAX_IMAGE_COUNT, len(ds.images))

keys = list(ds.images.keys())
keys = random.sample(keys, n)

box_annotator = sv.BoxAnnotator()

images = []
titles = []

for key in keys:
    frame_with_annotations = box_annotator.annotate(
        scene=ds.images[key].copy(),
        detections=ds.annotations[key],
        skip_label=True
    )
    images.append(frame_with_annotations)
    titles.append('annotations')
    frame_with_predictions = box_annotator.annotate(
        scene=ds.images[key].copy(),
        detections=predictions[key],
        skip_label=True
    )
    images.append(frame_with_predictions)
    titles.append('predictions')

%matplotlib inline
sv.plot_images_grid(images=images, titles=titles, grid_size=(n, 2), size=(2 * 4, n * 4))

### Calculate confusion matrix

In [ ]:
!pip install onemetric

In [ ]:
import os

import numpy as np

from onemetric.cv.object_detection import ConfusionMatrix

keys = list(ds.images.keys())

annotation_batches, prediction_batches = [], []

for key in keys:
    annotation=ds.annotations[key]
    annotation_batch = np.column_stack((
        annotation.xyxy,
        annotation.class_id
    ))
    annotation_batches.append(annotation_batch)

    prediction=predictions[key]
    prediction_batch = np.column_stack((
        prediction.xyxy,
        prediction.class_id,
        prediction.confidence
    ))
    prediction_batches.append(prediction_batch)

confusion_matrix = ConfusionMatrix.from_detections(
    true_batches=annotation_batches,
    detection_batches=prediction_batches,
    num_classes=len(ds.classes),
    conf_threshold=CONFIDENCE_TRESHOLD
)

confusion_matrix.plot(os.path.join(HOME, "confusion_matrix.png"), class_names=ds.classes)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save('/content/drive/MyDrive/saved_models/coin_counter.hdf5')

AttributeError: ignored

In [ ]:
path = F"/content/drive/MyDrive/saved_models/{EXPERIMENT_NAME}"
torch.save(model.state_dict(), path)

In [ ]:
model_save_name = 'cc.pt'
path = F"/content/drive/MyDrive/saved_models/{EXPERIMENT_NAME}"
model.load_state_dict(torch.load(path))

<All keys matched successfully>